# 妞妞

# import package

In [10]:
# import package
import itertools
import pandas as pd
import random
from collections import Counter

# 生成一副撲克牌

In [2]:
# def poker with rank & suit
suits = ['spade', 'heart', 'diamond', 'club']
ranks = ['A'] + [str(i) for i in range(2, 11)] + ['J', 'Q', 'K']

# generate a poker deck
deck = [(suit, rank) for suit in suits for rank in ranks]

# 轉換牌 & 排序大小

## 點數

In [3]:
# get card value
def card_value(card):
    # J, Q, K == 10
    # A == 1
    rank = card[1]
    return 10 if rank in ['J', 'Q', 'K'] else (1 if rank == 'A' else int(rank))

# get rank
def card_rank(card):
    rank_order = {str(i): i for i in range(2, 11)}
    rank_order.update({'J': 11, 'Q': 12, 'K': 13, 'A': 1})
    return rank_order[card[1]]

## 花色

In [4]:
# get suit value
def card_suit_value(card):
    suit_order = {'spade': 4, 'heart': 3, 'diamond': 2, 'club': 1}
    return suit_order[card[0]]

# 計算 5 張牌的牌型

In [5]:
# caculate fix 5 card type
def calculate_niu(cards):
    # compare value rank then suit rank
    cards_value = [card_value(card) for card in cards]
    cards.sort(key=lambda x: (card_rank(x), card_suit_value(x)), reverse=True)

    # sepcial card type
    if all(card[1] in ['J', 'Q', 'K'] for card in cards):
        return '五公', 5
    if all(v < 5 for v in cards_value) and sum(cards_value) == 10:
        return '五小牛', 6
    if sum(1 for card in cards if card[1] in ['J', 'Q', 'K']) == 4 and any(card[1] == '10' for card in cards):
        return '四花牛', 4

    # normal card type
    for combo in itertools.combinations(cards_value, 3):
        if sum(combo) % 10 == 0:
            remaining = [v for v in cards_value if v not in combo][:2]
            if sum(remaining) % 10 == 0:
                return '牛牛', 3
            else:
                remaining_score = sum(remaining) % 10
                if remaining_score >= 7:
                    return f'牛{remaining_score}', 2
                elif remaining_score > 0:
                    return f'牛{remaining_score}', 1

    # check J, Q, K
    has_face_card = any(card[1] in ['J', 'Q', 'K'] for card in cards)
    if has_face_card:
        return '有公無牛', 1
    else:
        return '無公無牛', -2


## 確認目前為止是否正確
1. 是否可以得到正確的牌型 <br>
測試出幾乎所有的情況

In [6]:
# hands
hand1 = [('spade', 'A'), ('heart', '2'), ('diamond', '3'), ('club', '2'), ('spade', '2')]
hand2 = [('spade', 'A'), ('heart', '2'), ('diamond', '3'), ('club', '2'), ('spade', '5')]
hand3 = [('spade', 'J'), ('heart', 'Q'), ('diamond', 'K'), ('club', 'J'), ('spade', 'Q')]
hand4 = [('spade', 'J'), ('heart', 'Q'), ('diamond', 'K'), ('club', 'J'), ('spade', '10')]
hand5 = [('spade', 'J'), ('heart', 'Q'), ('diamond', 'A'), ('club', '5'), ('spade', '8')]
hand6 = [('spade', 'A'), ('heart', '2'), ('diamond', '3'), ('club', '8'), ('spade', '3')]
hand7 = [('spade', 'J'), ('heart', 'Q'), ('diamond', '10'), ('club', 'J'), ('spade', '10')]
hand8 = [('spade', 'J'), ('heart', 'Q'), ('diamond', '10'), ('club', 'J'), ('spade', '9')]
hand9 = [('spade', 'J'), ('heart', 'Q'), ('diamond', '10'), ('club', 'J'), ('spade', '8')]
hand10 = [('spade', 'J'), ('heart', 'Q'), ('diamond', '10'), ('club', 'J'), ('spade', '7')]
hand11 = [('spade', 'J'), ('heart', 'Q'), ('diamond', '10'), ('club', 'J'), ('spade', '5')]

# result
print(calculate_niu(hand1))
print(calculate_niu(hand2))
print(calculate_niu(hand3))
print(calculate_niu(hand4))
print(calculate_niu(hand5))
print(calculate_niu(hand6))
print(calculate_niu(hand7))
print(calculate_niu(hand8))
print(calculate_niu(hand9))
print(calculate_niu(hand10))
print(calculate_niu(hand11))


('五小牛', 6)
('牛1', 1)
('五公', 5)
('四花牛', 4)
('有公無牛', 1)
('無公無牛', -2)
('牛牛', 3)
('牛9', 2)
('牛8', 2)
('牛7', 2)
('牛5', 1)


# 模擬牌型出現機率

In [45]:
# def simulate card type rate
def simulate_cardtype_rate(hand, num_simulations=100000):
    # generate poker deck
    suits = ['spade', 'heart', 'diamond', 'club']
    ranks = [str(i) for i in range(2, 11)] + ['J', 'Q', 'K', 'A']
    deck = [(suit, rank) for suit in suits for rank in ranks]

    # remove cards already in hand
    for card in hand:
        deck.remove(card)

    results = Counter()
    for _ in range(num_simulations):
        # random choose a card
        remaining_card = random.choice(deck)
        final_hand = hand + [remaining_card]
        # only want to know card type
        card_type, _ = calculate_niu(final_hand)
        results[card_type] += 1

    total = sum(results.values())
    probabilities = {k: v / total for k, v in results.items()}
    sorted_probabilities = dict(sorted(probabilities.items(), key=lambda item: item[1], reverse=True))
    return sorted_probabilities


## 確認機率情況

In [46]:
hand1 = [('spade', 'J'), ('heart', 'Q'), ('diamond', '3'), ('club', '2')]
simulate_cardtype_rate(hand1)

{'有公無牛': 0.45886, '牛5': 0.29242, '牛2': 0.08345, '牛3': 0.0829, '牛牛': 0.08237}

# 結算金額

In [ ]:
def get_card_value(card):
    """取得牌的點數大小，K 最大 (13)，A 最小 (1)"""
    rank_order = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10,
                  'J': 11, 'Q': 12, 'K': 13, 'A': 1}
    return rank_order[card[1]]

def get_suit_value(card):
    """取得牌的花色大小，spade > heart > diamond > club"""
    suit_order = {'spade': 4, 'heart': 3, 'diamond': 2, 'club': 1}
    return suit_order[card[0]]

def compare_highest_card(player_hand, banker_hand):
    """比較玩家與莊家最大的牌，先比數字，再比花色"""
    player_hand_sorted = sorted(player_hand, key=lambda x: (get_card_value(x), get_suit_value(x)), reverse=True)
    banker_hand_sorted = sorted(banker_hand, key=lambda x: (get_card_value(x), get_suit_value(x)), reverse=True)
    
    player_max_card = player_hand_sorted[0]
    banker_max_card = banker_hand_sorted[0]

    if get_card_value(player_max_card) > get_card_value(banker_max_card):
        return "player"
    elif get_card_value(player_max_card) < get_card_value(banker_max_card):
        return "banker"
    else:  # 如果點數一樣，則比花色
        if get_suit_value(player_max_card) > get_suit_value(banker_max_card):
            return "player"
        else:
            return "banker"

def calculate_payout(player_type, banker_type, player_hand, banker_hand):
    """根據玩家和莊家的牌型與最大牌計算結算金額"""
    rank_mult = {
        '五小牛': 6, '五公': 5, '四花牛': 4, '牛牛': 3, '牛9': 2, '牛8': 2,
        '牛7': 2, '牛6': 1, '牛5': 1, '牛4': 1, '牛3': 1, '牛2': 1, '牛1': 1,
        '有公無牛': 1, '無公無牛': 2
    }
    rank_order = {
        '五小牛': 15, '五公': 14, '四花牛': 13, '牛牛': 12, '牛9': 11, '牛8': 10,
        '牛7': 9, '牛6': 8, '牛5': 7, '牛4': 6, '牛3': 5, '牛2': 4, '牛1': 3,
        '有公無牛': 2, '無公無牛': 1
    }

    player_rank_order = rank_order[player_type]
    banker_rank_order = rank_order[banker_type]
    player_rank_mult = rank_mult[player_type]
    banker_rank_mult = rank_mult[banker_type]

    if player_rank_order > banker_rank_order:
        multiplier = player_rank_mult if banker_type != '無公無牛' else player_rank_mult * 2
        return f"玩家贏，賠率為 {multiplier} 倍"
    
    elif player_rank_order < banker_rank_order:
        multiplier = banker_rank_mult * (-1) if player_type != '無公無牛' else banker_rank_mult * -2
        return f"玩家輸，賠率為 {multiplier} 倍"

    else:  # 當牌型相同時，進一步比較最大牌
        winner = compare_highest_card(player_hand, banker_hand)
        if winner == "player":
            multiplier = player_rank_mult if banker_type != '無公無牛' else player_rank_mult * 2
            return f"玩家贏，賠率為 {multiplier} 倍"
        else:
            multiplier = banker_rank_mult * (-1) if player_type != '無公無牛' else banker_rank_mult * -2
            return f"玩家輸，賠率為 {multiplier} 倍"


In [37]:
def calculate_payout(player_type, banker_type):
    """根據玩家和莊家的牌型計算結算金額"""
    rank_mult = {
        '五小牛': 6, '五公': 5, '四花牛': 4, '牛牛': 3, '牛9': 2, '牛8': 2,
        '牛7': 2, '牛6': 1, '牛5': 1, '牛4': 1, '牛3': 1, '牛2': 1, '牛1': 1,
        '有公無牛': 1, '無公無牛': 2
    }
    rank_order = {
        '五小牛': 15, '五公': 14, '四花牛': 13, '牛牛': 12, '牛9': 11, '牛8': 10,
        '牛7': 9, '牛6': 8, '牛5': 7, '牛4': 6, '牛3': 5, '牛2': 4, '牛1': 3,
        '有公無牛': 2, '無公無牛': 1
    }

    player_rank_order = rank_order[player_type]
    bank_rank_order = rank_order[banker_type]
    player_rank_mult = rank_mult[player_type]
    banker_rank_mult = rank_mult[banker_type]

    if player_rank_order > bank_rank_order:
        # 如果莊家是無公無牛，則玩家額外獲得 2 倍賠率
        multiplier = player_rank_mult if banker_type != '無公無牛' else player_rank_mult *2
        return f"玩家贏，賠率為 {multiplier} 倍"
    
    elif player_rank_order < bank_rank_order:
        # 如果玩家是無公無牛，則莊家額外獲得 2 倍賠率
        multiplier = banker_rank_mult * (-1) if player_type != '無公無牛' else banker_rank_mult *-2
        return f"玩家輸，賠率為 {multiplier} 倍"

In [38]:
calculate_payout("牛2", "牛1")

'玩家贏，賠率為 1 倍'

# 進行預估

In [ ]:
def recommend_bet(hand):
    """根據機率推薦下注倍率"""
    probabilities = simulate_winrate(hand)

    # 確保牌型大小排序
    rank_order = [
        '五小牛', '五公', '四花牛', '牛牛', '牛九', '牛八', '牛七',
        '牛六', '牛五', '牛四', '牛三', '牛二', '牛一', '有公無牛', '無公無牛'
    ]

    # 計算高獎勵牌型的總機率
    high_reward_types = rank_order[:7]  # 前 7 種牌型
    high_reward_prob = sum(probabilities.get(t, 0) for t in high_reward_types)

    if high_reward_prob > 0.5:
        return "推薦下注 5 倍，因為高獎勵機率高"
    elif high_reward_prob > 0.3:
        return "推薦下注 3 倍，風險與收益適中"
    else:
        return "推薦下注 1 倍，穩健策略"

# 實際操作

In [41]:

# 假設手牌
hand = ['J', 'J', 'Q', '10']
print("手牌為:", hand)

# 模擬贏率
probabilities = simulate_winrate(hand)
print("模擬結果:")
for card_type, prob in probabilities.items():
    print(f"{card_type}: {prob * 100:.2f}%")


手牌為: ['J', 'J', 'Q', '10']


ValueError: list.remove(x): x not in list

In [131]:
def recommend_bet(hand):
    """根據機率推薦下注倍率"""
    probabilities = simulate_winrate(hand)

    # 確保牌型大小排序
    rank_order = [
        '五小牛', '五公', '四花牛', '牛牛', '牛九', '牛八', '牛七',
        '牛六', '牛五', '牛四', '牛三', '牛二', '牛一', '有公無牛', '無公無牛'
    ]

    # 計算高獎勵牌型的總機率
    high_reward_types = rank_order[:7]  # 前 7 種牌型
    high_reward_prob = sum(probabilities.get(t, 0) for t in high_reward_types)

    if high_reward_prob > 0.5:
        return "推薦下注 5 倍，因為高獎勵機率高"
    elif high_reward_prob > 0.3:
        return "推薦下注 3 倍，風險與收益適中"
    else:
        return "推薦下注 1 倍，穩健策略"

def recommend_banker_bet(hand, num_simulations=10000):
    """根據手牌模擬是否應該搶莊以及搶莊倍數"""
    deck = [str(i) for i in range(1, 11)] + ['J', 'Q', 'K']
    deck *= 4

    # 移除手牌中的卡片
    for card in hand:
        deck.remove(card)

    win_counts = {1: 0, 2: 0, 3: 0, 4: 0}

    for _ in range(num_simulations):
        # 模擬其他玩家的手牌
        other_hands = [random.sample(deck, 5) for _ in range(3)]

        # 計算其他玩家的最佳牌型
        other_types = [calculate_niu(hand)[0] for hand in other_hands]

        # 判斷當前玩家的牌型
        final_hand = hand + [random.choice(deck)]
        player_type, _ = calculate_niu(final_hand)

        # 模擬莊家的勝率
        for multiple in [1, 2, 3, 4]:
            if all(player_type >= t for t in other_types):
                win_counts[multiple] += 1

    # 決定最適合的搶莊倍數
    best_multiple = max(win_counts, key=win_counts.get)
    return f"推薦搶莊 {best_multiple} 倍"

# 使用示例
if __name__ == "__main__":
    # 假設手牌
    hand = ['1', 'J', '3', '10']
    print("手牌為:", hand)

    # 模擬贏率
    probabilities = simulate_winrate(hand)
    print("模擬結果:")
    for card_type, prob in probabilities.items():
        print(f"{card_type}: {prob * 100:.2f}%")

    # 推薦下注策略
    recommendation = recommend_bet(hand)
    print("下注建議:", recommendation)

    # 推薦搶莊策略
    banker_recommendation = recommend_banker_bet(hand)
    print("搶莊建議:", banker_recommendation)

    # 結算範例
    player_type = '牛九'
    banker_type = '牛牛'
    payout = calculate_payout(player_type, banker_type)
    print(f"結算結果: {payout}")


手牌為: ['1', 'J', '3', '10']
模擬結果:
牛4: 29.26%
無公無牛: 45.58%
牛1: 8.50%
牛牛: 8.06%
牛3: 8.60%
下注建議: 推薦下注 1 倍，穩健策略
搶莊建議: 推薦搶莊 1 倍
結算結果: 玩家輸，賠率為 1 倍


In [17]:



def compare_hands(hand1, hand2):
    """比較兩手牌的大小，返回獲勝手牌"""
    # 檢查兩手牌是否有重複的牌
    if not set(hand1).isdisjoint(set(hand2)):  # 如果 hand1 和 hand2 之間有交集
        return "不可出現重複手牌", None
    
    type1 = calculate_niu(hand1)[0]
    type2 = calculate_niu(hand2)[0]
    
    rank_order = [
        '五小牛', '五公', '四花牛', '牛牛', '牛9', '牛8', '牛7',
        '牛6', '牛5', '牛4', '牛3', '牛2', '牛1', '有公無牛', '無公無牛'
    ]
    
    if rank_order.index(type1) < rank_order.index(type2):
        return hand1, type1
    elif rank_order.index(type1) > rank_order.index(type2):
        return hand2, type2
    elif rank_order.index(type1) == rank_order.index(type2):
        # 兩手牌的牌型相同時，先比較最前面那張牌的 rank 再比較花色
        hand1_sorted = sorted(hand1, key=lambda x: (card_rank(x), card_suit_value(x)), reverse=True)
        hand2_sorted = sorted(hand2, key=lambda x: (card_rank(x), card_suit_value(x)), reverse=True)

        for card1, card2 in zip(hand1_sorted, hand2_sorted):
            if card_rank(card1) > card_rank(card2):
                return hand1, type1
            elif card_rank(card1) < card_rank(card2):
                return hand2, type2
            else:
                if card_suit_value(card1) > card_suit_value(card2):
                    return hand1, type1
                elif card_suit_value(card1) < card_suit_value(card2):
                    return hand2, type2
        return hand1, type1  # 如果所有牌相同，返回手牌1

# 測試比較兩手牌
if __name__ == "__main__":
    hand1 = [('spade', '8'), ('heart', '5'), ('diamond', '5'), ('club', '8'), ('heart', 'K')]
    hand2 = [('spade', 'K'), ('heart', 'Q'), ('diamond', 'J'), ('club', '10'), ('spade', '6')]
    print(compare_hands(hand1, hand2))
    
    # 測試有重複牌的情況 (兩手牌有重複的牌)
    hand3 = [('spade', '8'), ('heart', '5'), ('diamond', '5'), ('club', '8'), ('spade', '10')]
    hand4 = [('spade', '8'), ('heart', '5'), ('diamond', '5'), ('club', '8'), ('spade', '10')]
    print(compare_hands(hand3, hand4))  # 這會顯示「不可出現重複手牌」
    
    # 測試其中一手牌有重複
    hand5 = [('spade', '8'), ('heart', '5'), ('diamond', '5'), ('club', '8'), ('spade', '8')]
    print(compare_hands(hand5, hand2))  # 這會顯示「不可出現重複手牌」


([('spade', 'K'), ('heart', 'Q'), ('diamond', 'J'), ('club', '10'), ('spade', '6')], '牛6')
('不可出現重複手牌', None)
([('spade', 'K'), ('heart', 'Q'), ('diamond', 'J'), ('club', '10'), ('spade', '6')], '牛6')
